In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt


In [3]:

def get_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    data['Date'] = data.index
    return data

def get_data_from_csv(ticker):
    return pd.read_csv(f'nifty_200/{ticker}.csv')

In [5]:
data = get_data_from_csv("ABB")

In [10]:

def identify_inside_bars(df):
    df['InsideBar'] = (
        (df[['Close', 'Open']].max(axis=1) < df[['Close', 'Open']].shift(1).max(axis=1)) &
        (df[['Close', 'Open']].min(axis=1) > df[['Close', 'Open']].shift(1).min(axis=1))
    )
    return df
def identify_consecutive_inside_bars(df):
    # Initialize the InsideBar column as False
    df['InsideBar'] = False
    
    # Iterate through the DataFrame starting from the second row
    for i in range(1, len(df)):
        # Check if the current bar is an inside bar relative to the previous bar
        if (df.iloc[i]['High'] < df.iloc[i - 1]['High']) and (df.iloc[i]['Low'] > df.iloc[i - 1]['Low']):
            df.at[i, 'InsideBar'] = True
    
    # Create a new column to count consecutive inside bars
    df['ConsecutiveInsideBars'] = 0
    consecutive_count = 0
    
    for i in range(1, len(df)):
        if df.at[i, 'InsideBar']:
            consecutive_count += 1
        else:
            consecutive_count = 0
        df.at[i, 'ConsecutiveInsideBars'] = consecutive_count
    
    return df

def calculate_emas(df, emas):
    for len in emas:
        df[f"EMA_{len}"] = df['Close'].ewm(span=len, adjust=False).mean()
    return df

def apply_strategy(df):
    df['Buy_Signal'] = np.where(df['InsideBar'] & (df['Close'] > df['High'].shift(1)), 1, 0)
    df['Sell_Signal'] = np.where(df['InsideBar'] & (df['Close'] < df['Low'].shift(1)), -1, 0)
    return df

def backtest_strategy(df, position_size):
    df['Position'] = df['Buy_Signal'] + df['Sell_Signal']
    df['Position'] = df['Position'].shift(1).fillna(0)
    
    df['Market_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Strategy_Return'] = df['Position'] * df['Market_Return']
    
    df['Trade_Return'] = df['Strategy_Return'] * position_size
    df['Cumulative_Trade_Return'] = df['Trade_Return'].cumsum()

    total_return = df['Cumulative_Trade_Return'].iloc[-1]
    num_trades = df['Position'].abs().sum()
    win_rate = (df['Trade_Return'] > 0).sum() / num_trades if num_trades > 0 else 0
    max_drawdown = df['Cumulative_Trade_Return'].max() - df['Cumulative_Trade_Return'].min()
    sharpe_ratio = df['Strategy_Return'].mean() / df['Strategy_Return'].std() * np.sqrt(252) if df['Strategy_Return'].std() > 0 else 0
    
    kpis = {
        'Total Return': total_return,
        'Number of Trades': num_trades,
        'Win Rate': win_rate,
        'Max Drawdown': max_drawdown,
        'Sharpe Ratio': sharpe_ratio
    }
    
    return kpis, df

def plot_results(df, ticker):
    plt.figure(figsize=(12, 8))
    plt.plot(df['Cumulative_Trade_Return'], label=f'{ticker} Strategy Return')
    plt.title(f'Inside Bar Trading Strategy Backtest for {ticker}')
    plt.legend()
    plt.show()

def backtest_multiple_stocks(tickers, start_date, end_date, position_size):
    all_kpis = []

    for ticker in tickers:
        data = get_data(ticker, start_date, end_date)
        data = identify_inside_bars(data)
        data = calculate_emas(data, emas) 
        data = apply_strategy(data)
        kpis, backtested_data = backtest_strategy(data, position_size)

        kpis['Ticker'] = ticker
        all_kpis.append(kpis)

        plot_results(backtested_data, ticker)

    return pd.DataFrame(all_kpis)


In [ ]:
df = identify_inside_bars(data)

In [12]:
consecutive_inside_bars_df = identify_consecutive_inside_bars(data)

In [16]:
consecutive_inside_bars_df[consecutive_inside_bars_df['ConsecutiveInsideBars'] > 2]

,Datetime,Open,High,Low,Close,Volume,InsideBar,ConsecutiveInsideBars
909,2021-01-06 12:15:00,1311.35,1311.90,1309.00,1310.85,16708,True,3
1216,2021-03-10 13:15:00,1485.50,1489.85,1483.25,1486.75,12781,True,3
1379,2021-04-16 15:15:00,1370.35,1372.50,1370.25,1371.35,5860,True,3
2358,2021-11-10 12:15:00,2198.00,2198.85,2192.25,2195.00,22536,True,3
5076,2023-06-06 13:15:00,4053.40,4060.00,4050.55,4057.75,11414,True,3
5173,2023-06-26 12:15:00,4255.50,4259.50,4252.80,4256.10,7907,True,3
5299,2023-07-21 12:15:00,4283.05,4287.90,4260.50,4267.25,23185,True,3
5544,2023-09-11 12:15:00,4622.00,4645.10,4622.00,4642.70,25029,True,3
5810,2023-11-07 12:15:00,4200.00,4204.60,4195.35,4201.20,3978,True,3
6381,2024-03-05 12:15:00,5642.90,5662.00,5631.15,5647.10,21624,True,3


In [14]:
data.iloc[0]

Datetime                 2020-07-02 09:15:00
Open                                   966.0
High                                   975.0
Low                                    955.1
Close                                 963.25
Volume                                 43559
InsideBar                              False
ConsecutiveInsideBars                      0
Name: 0, dtype: object

In [9]:
historical_data = data
# Assuming `historical_data` is your DataFrame containing the historical data

# Initialize lists and variables for the backtest
inside_bars = []
consecutive_inside_bars = 0  # Initialize a counter for consecutive inside bars
results = []
position = None

# Backtest parameters
stop_loss_multiple = 1.0
take_profit_multiple = 2.0

# Iterate through historical data to detect inside bars and apply the trading strategy
for i in range(1, len(historical_data)):
    current_candle = historical_data.iloc[i]
    previous_candle = historical_data.iloc[i - 1]

    # Detect inside bars
    if (current_candle['High'] < previous_candle['High']) and (current_candle['Low'] > previous_candle['Low']):
        inside_bars.append(current_candle)
        consecutive_inside_bars += 1
    else:
        consecutive_inside_bars = 0  # Reset the counter if no inside bar

    # Execute strategy on triple inside bar breakout
    if consecutive_inside_bars >= 3:
        print(f'The multiple inside bars detected on {current_candle['Datetime']}')
        entry_high = max(candle['High'] for candle in inside_bars)
        entry_low = min(candle['Low'] for candle in inside_bars)
        
        if position is None:
            # Long position
            if current_candle['Close'] > entry_high:
                stop_loss = entry_low - stop_loss_multiple * (entry_high - entry_low)
                take_profit = entry_high + take_profit_multiple * (entry_high - entry_low)
                results.append({'Datetime': current_candle['Datetime'], 'Type': 'Buy', 'Entry': entry_high,
                                'Stop Loss': stop_loss, 'Take Profit': take_profit, 'Outcome': None})
                position = 'Long'
            # Short position
            elif current_candle['Close'] < entry_low:
                stop_loss = entry_high + stop_loss_multiple * (entry_high - entry_low)
                take_profit = entry_low - take_profit_multiple * (entry_high - entry_low)
                results.append({'Datetime': current_candle['Datetime'], 'Type': 'Sell', 'Entry': entry_low,
                                'Stop Loss': stop_loss, 'Take Profit': take_profit, 'Outcome': None})
                position = 'Short'
        
        # Reset the consecutive inside bar counter
        consecutive_inside_bars = 0

    # Manage position
    if position == 'Long':
        if current_candle['Low'] <= stop_loss:
            results[-1]['Outcome'] = 'Loss'
            position = None
        elif current_candle['High'] >= take_profit:
            results[-1]['Outcome'] = 'Profit'
            position = None
    elif position == 'Short':
        if current_candle['High'] >= stop_loss:
            results[-1]['Outcome'] = 'Loss'
            position = None
        elif current_candle['Low'] <= take_profit:
            results[-1]['Outcome'] = 'Profit'
            position = None

# Convert results to a DataFrame for analysis
backtest_results = pd.DataFrame(results)

# Display the results
print(backtest_results.head())


The multiple inside bars detected on 2021-01-06 12:15:00
The multiple inside bars detected on 2021-03-10 13:15:00
The multiple inside bars detected on 2021-04-16 15:15:00
The multiple inside bars detected on 2021-11-10 12:15:00
The multiple inside bars detected on 2023-06-06 13:15:00
The multiple inside bars detected on 2023-06-26 12:15:00
The multiple inside bars detected on 2023-07-21 12:15:00
The multiple inside bars detected on 2023-09-11 12:15:00
The multiple inside bars detected on 2023-11-07 12:15:00
The multiple inside bars detected on 2024-03-05 12:15:00
Empty DataFrame
Columns: []
Index: []


In [ ]:
df[df['InsideBar']]

,Open,High,Low,Close,Adj Close,Volume,InsideBar
Datetime,,,,,,,
2022-08-10 14:15:00+05:30,514.799988,515.900024,513.700012,515.099976,515.099976,3508643,True
2022-08-11 12:15:00+05:30,520.299988,521.599976,520.150024,521.299988,521.299988,838146,True
2022-08-12 10:15:00+05:30,529.900024,530.250000,528.150024,529.750000,529.750000,1336419,True
2022-08-12 11:15:00+05:30,529.799988,530.400024,529.150024,529.799988,529.799988,1089727,True
2022-08-16 14:15:00+05:30,527.500000,528.000000,525.599976,526.400024,526.400024,1878918,True
...,...,...,...,...,...,...,...
2024-07-23 14:15:00+05:30,865.799988,871.750000,862.049988,863.450012,863.450012,3184395,True
2024-07-24 11:15:00+05:30,853.599976,855.750000,849.200012,854.849976,854.849976,2585772,True
2024-07-26 13:15:00+05:30,859.299988,862.700012,857.849976,859.500000,859.500000,1706221,True


In [ ]:

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
position_size = 100000
start_date = '2020-01-01'
end_date = '2024-08-09'

kpi_df = backtest_multiple_stocks(tickers, start_date, end_date, position_size)
print(kpi_df)

print(kpi_df)

# Save KPIs to a file
kpi_df.to_csv('inside_bar_strategy_kpis.csv', index=False)


In [26]:
import pandas as pd


def identify_inside_bars(df):
    df['Inside Bar'] = (
        (df[['Close', 'Open']].max(axis=1) < df[['Close', 'Open']].shift(1).max(axis=1)) &
        (df[['Close', 'Open']].min(axis=1) > df[['Close', 'Open']].shift(1).min(axis=1))
    )
    return df


def identify_consecutive_inside_bars(df):
    df['InsideBar'] = False
    
    for i in range(1, len(df)):
        if (df.iloc[i]['High'] < df.iloc[i - 1]['High']) and (df.iloc[i]['Low'] > df.iloc[i - 1]['Low']):
            df.at[i, 'InsideBar'] = True
    
    df['ConsecutiveInsideBars'] = 0
    consecutive_count = 0
    
    for i in range(1, len(df)):
        if df.at[i, 'InsideBar']:
            consecutive_count += 1
        else:
            consecutive_count = 0
        df.at[i, 'ConsecutiveInsideBars'] = consecutive_count
    
    return df

def get_mother_bar(df, index, consecutive_bars):
    """Returns the mother bar for the given index."""
    return df.iloc[index - consecutive_bars]

def calculate_entry_levels(mother_bar):
    """Calculates the entry high and low based on the mother bar."""
    entry_high = mother_bar['High']
    entry_low = mother_bar['Low']
    return entry_high, entry_low

def calculate_stop_loss_and_take_profit(entry_high, entry_low, stop_loss_multiple, take_profit_multiple, position_type):
    """Calculates the stop loss and take profit levels based on the position type (Long/Short)."""
    if position_type == 'Long':
        stop_loss = entry_low - stop_loss_multiple * (entry_high - entry_low)
        take_profit = entry_high + take_profit_multiple * (entry_high - entry_low)
    elif position_type == 'Short':
        stop_loss = entry_high + stop_loss_multiple * (entry_high - entry_low)
        take_profit = entry_low - take_profit_multiple * (entry_high - entry_low)
    return stop_loss, take_profit

def manage_position(current_candle, stop_loss, take_profit, position):
    """Manages the open position and determines if it hits stop loss or take profit."""
    if position == 'Long':
        if current_candle['Low'] <= stop_loss:
            return 'Loss', None
        elif current_candle['High'] >= take_profit:
            return 'Profit', None
    elif position == 'Short':
        if current_candle['High'] >= stop_loss:
            return 'Loss', None
        elif current_candle['Low'] <= take_profit:
            return 'Profit', None
    return None, position

def backtest_inside_bar_strategy(df, required_consecutive_bars=2, stop_loss_multiple=1.0, take_profit_multiple=2.0):
    df = identify_consecutive_inside_bars(df)
    df = identify_inside_bars(df)
    
    results = []
    position = None
    stop_loss = None  # Initialize stop_loss
    take_profit = None  # Initialize take_profit
    
    for i in range(1, len(df)):
        current_candle = df.iloc[i]
        
        if current_candle['ConsecutiveInsideBars'] >= required_consecutive_bars:
            mother_bar = get_mother_bar(df, i, required_consecutive_bars)
            entry_high, entry_low = calculate_entry_levels(mother_bar)
            
            if position is None:
                if df.iloc[i + 1]['Close'] > entry_high :
                    stop_loss, take_profit = calculate_stop_loss_and_take_profit(entry_high, entry_low, stop_loss_multiple, take_profit_multiple, 'Long')
                    results.append({'Datetime': df.iloc[i + 1]['Datetime'], 'Type': 'Buy', 'Entry': entry_high,
                                    'Stop Loss': stop_loss, 'Take Profit': take_profit, 'Outcome': None})
                    position = 'Long'
                elif df.iloc[i + 1]['Close'] < entry_low:
                    stop_loss, take_profit = calculate_stop_loss_and_take_profit(entry_high, entry_low, stop_loss_multiple, take_profit_multiple, 'Short')
                    results.append({'Datetime': df.iloc[i + 1]['Datetime'], 'Type': 'Sell', 'Entry': entry_low,
                                    'Stop Loss': stop_loss, 'Take Profit': take_profit, 'Outcome': None})
                    position = 'Short'
        
        # Manage the open position if it exists
        if position is not None:
            outcome, position = manage_position(current_candle, stop_loss, take_profit, position)
            if outcome:
                results[-1]['Outcome'] = outcome
    
    return pd.DataFrame(results) , df

# Example usage:
# Backtest with the requirement of 2 consecutive inside bars, using the high and low of the mother bar
backtest_results, bt_df = backtest_inside_bar_strategy(historical_data, required_consecutive_bars=2)


In [24]:

# Display the backtest results
backtest_results


,Datetime,Open,High,Low,Close,Volume,InsideBar,ConsecutiveInsideBars
0,2020-07-02 09:15:00,966.00,975.00,955.10,963.25,43559,False,0
1,2020-07-02 10:15:00,963.25,965.40,957.00,963.00,39929,True,1
2,2020-07-02 11:15:00,963.00,967.00,958.60,967.00,9104,False,0
3,2020-07-02 12:15:00,967.00,968.40,959.90,962.55,9672,False,0
4,2020-07-02 13:15:00,962.50,973.75,961.10,973.70,17527,False,0
...,...,...,...,...,...,...,...,...
7111,2024-08-09 11:15:00,8018.05,8036.85,7921.00,8004.45,164059,False,0
7112,2024-08-09 12:15:00,8005.00,8038.20,7980.60,8005.80,96145,False,0
7113,2024-08-09 13:15:00,8005.80,8013.65,7950.70,7994.10,86556,False,0
7114,2024-08-09 14:15:00,7991.40,7999.00,7954.75,7994.60,100328,True,1


In [27]:


bt_df[bt_df['Inside Bar']]

,Datetime,Open,High,Low,Close,Volume,InsideBar,ConsecutiveInsideBars,Inside Bar
15,2020-07-06 10:15:00,1010.55,1012.15,992.45,998.55,103625,True,1,True
19,2020-07-06 14:15:00,983.95,990.60,981.05,984.00,40907,True,1,True
27,2020-07-07 15:15:00,960.00,963.50,958.80,961.40,23864,False,0,True
50,2020-07-13 10:15:00,917.05,919.75,915.50,916.70,6494,True,1,True
62,2020-07-14 15:15:00,907.35,912.95,906.30,908.15,3525,False,0,True
...,...,...,...,...,...,...,...,...,...
7070,2024-08-01 12:15:00,7781.15,7814.95,7780.00,7801.50,15466,False,0,True
7080,2024-08-02 15:15:00,7576.90,7592.05,7565.00,7574.00,17503,True,1,True
7094,2024-08-06 15:15:00,7447.95,7501.20,7428.00,7460.00,21822,False,0,True
7108,2024-08-08 15:15:00,7932.45,7987.95,7910.00,7950.00,32710,False,0,True
